In [1]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from cnn import ConvNet 
import cv2
import argparse
import numpy as np     
import matplotlib.pyplot as plt
from cProfile import label
import json
import sys
# import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check if cuda is available
use_cuda = torch.cuda.is_available()

# Set proper device based on cuda availability 
device = torch.device("cuda" if use_cuda else "cpu")
print("Torch device selected: ", device)

# Create transformations to apply to each data sample 
# Can specify variations such as image flip, color flip, random crop, ...
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.0,), (1.0,))
        ])

# Load datasets for training and testing
# Inbuilt datasets available in torchvision (check documentation online)
dataset1 = datasets.CIFAR10('./data/', train=True, download=True,
                    transform=transform)
dataset2 = datasets.CIFAR10('./data/', train=False,
                    transform=transform)

Torch device selected:  cpu
Files already downloaded and verified


In [3]:
def load_data(batch_size=10, num_workers=4):
    train_loader = DataLoader(dataset1, batch_size = batch_size, 
                            shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = batch_size, 
                                shuffle=False, num_workers=4)
    
    return train_loader, test_loader

def plot(num_epochs, losses, accuracies, best_accuracy, save=0, mode=1):
    x = range(1, num_epochs+1)

    train_accuracies, test_accuracies = accuracies[0], accuracies[1]
    train_losses, test_losses = losses[0], losses[1]
    ## accuracies plot
    plt.title(f'Model {mode} Accuracies')
    plt.plot(x, train_accuracies, 'r')
    plt.plot(x, test_accuracies, 'b')
    plt.legend(['Train Accuracy', 'Test Accuracy'])

    if save:
        plt.savefig(f'plots/model_{mode}_accuracies.jpg')
        plt.show()

    ## losses plot
    plt.title(f'Model {mode} Losses')
    plt.plot(x, train_losses, 'r')
    plt.plot(x, test_losses, 'b')
    plt.legend(['Train Losses', 'Test Losses'])

    if save:
        plt.savefig(f'plots/model_{mode}_losses.jpg')
        plt.show()

def train(model, device, train_loader, optimizer, criterion, epoch, batch_size, num_epochs):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        # if batch_idx == 3:
        #     break
        data, target = batch_sample
        # print(f'{data.shape = }')
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        # print(output.detach().numpy())
        # Compute loss based on criterion
        loss = criterion(output, target)
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        _, predictions = output.max(1)
        correct += (predictions == target).sum()
        print(f'Training epoch: ({epoch}/{num_epochs}) batch: ({batch_idx+1}/{len(train_loader)})', end='\r') #. Acc: {correct}/{(batch_idx+1) * batch_size}, {100. * correct / ((batch_idx+1) * batch_size)}', end='\r')
        
    train_loss = float(np.mean(losses))
    train_acc = 100. * (correct / ((batch_idx+1) * batch_size))
    print('\nTrain set ({}/{}): Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch, num_epochs,
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size, train_acc))
    return train_loss, train_acc
    
def test(model, device, test_loader, criterion, epoch, num_epochs, batch_size):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            # Predict for data by doing forward pass
            output = model(data)
        
            # Compute loss based on same criterion as training 
            loss = criterion(output, target)
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            _, predictions = output.max(1)
            correct += (predictions == target).sum()
            print(f'Testing epoch: ({epoch}/{num_epochs}) batch: ({batch_idx+1}/{len(test_loader)})', end='\r')

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set ({}/{}): Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch, num_epochs,
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy


In [4]:
def run_model(mode=1, learning_rate=0.01, batch_size=10, num_epochs=60):
    image_size = 32*32*3
    num_classes = 10

    # Initialize the model and send to device 
    model = ConvNet(mode, image_size, num_classes).to(device)
    # Define loss function.
    criterion = nn.CrossEntropyLoss()
    # Define optimizer function.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # Define data loaders
    train_loader, test_loader = load_data(batch_size)

    best_accuracy = 0.0

    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    # Run training for n_epochs specified in config 
    for epoch in range(1, num_epochs + 1):
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, batch_size, num_epochs)
        test_loss, test_accuracy = test(model, device, test_loader, criterion, epoch, num_epochs, batch_size)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        # print(train_accuracy.cpu().numpy())
        train_accuracies.append(train_accuracy.cpu().numpy())
        test_accuracies.append(test_accuracy.cpu().numpy())

    accuracies = [train_accuracies, test_accuracies]
    losses = [train_losses, test_losses]
    plot(num_epochs, losses, accuracies, best_accuracy, save=1, mode=mode)

    print("Accuracy: {:2.2f}%".format(best_accuracy))

    print("Training and evaluation finished")

In [5]:
# ================== Model 1 ==================
learning_rate = 0.001
batch_size = 32
num_epochs = 30

print('\n\n'+('='*32)+' Training model 1 '+('='*32))
print('CNN architecture which has more than 2 conv layers (3) and more than 1 fully connected layers (2)')
print('\nlearning_rate = {}\nbatch_size = {}\nnum_epochs = {}\n'.format(learning_rate, batch_size, num_epochs))
run_model(mode=1, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)    
# print('='*80)

In [6]:
# ================== Model 2 ==================
learning_rate = 0.001
batch_size = 32
num_epochs = 30

print(('='*32)+' Training model 2 '+('='*32))
print('Increase the number of conv layers (5) in the above network and train again.')
print('\nlearning_rate = {}\nbatch_size = {}\nnum_epochs = {}\n'.format(learning_rate, batch_size, num_epochs))

run_model(mode=2, learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epochs)
print('='*80)

================================ Training model 2 ================================
Increase the number of conv layers (5) in the above network and train again.

learning_rate = 0.1
batch_size = 64
num_epochs = 30

Training epoch: (1/30) batch: (782/782)
Train set (1/30): Average loss: 2.3613, Accuracy: 4987/50048 (10%)

Testing epoch: (1/30) batch: (157/157)
Test set (1/30): Average loss: 2.3613, Accuracy: 1000/10000 (10%)



KeyboardInterrupt: 